In [ ]:
print( 'ФИО: Кирилл Живин' )

import os
import logging
import psycopg2
import psycopg2.extensions
from pymongo import MongoClient

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Создаём подключёние к Postgres")
params = {
    "host": os.environ['APP_POSTGRES_HOST'],
    "port": os.environ['APP_POSTGRES_PORT'],
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)

import datetime
from subprocess import call
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

df = pd.read_sql_query("select round ( avg(order_cart_sum) :: numeric , 0 ) as avg_order_sum from orders;", conn)

print(df)

df = pd.read_sql_query("select count(cart_order_id) as more_10_order from order_cart where pie_count>10;", conn)

print(df)

df = pd.read_sql_query("""select order_id from orders 
where order_delivery_price>=500 and order_source='internet' limit 10;""", conn)

print(df)

df = pd.read_sql_query("""select order_id, order_delivery_price, order_cart_sum,
    avg(order_cart_sum) over ( partition by order_delivery_price ) as avg_sum
from orders where order_delivery_price!= 0 order by order_delivery_price asc limit 10;""", conn)

print(df)

df = pd.read_sql_query(""" with A1 as (select person_id from order_persons group by person_id having count(order_id) >= 5)
select round( avg(sum_pie) :: numeric, 2 ) from ( select sum(pie_count) as sum_pie
from A1
join order_persons as OP on OP.person_id = A1.person_id
join order_cart as OC on OP.order_id = OC.cart_order_id group by OP.order_id ) A2; """, conn)

print(df)

df.to_csv("/home/pie.csv")


